In [1]:
# This file uses the inhouse qcbm code from the paper and compares MMD with MCR
import numpy as np
import matplotlib.pyplot as plt
import copy
import q_circuit as q # In house circuit code, to comare with Qiskit
import torch
import time
import os

# Oh Hail the almighty Qiskit:
import qiskit
from qiskit import transpile, assemble
from qiskit.visualization import *
from qiskit import IBMQ
from qiskit.providers.jobstatus import JobStatus

# Import custom packages: These are written by me
import InHouse_Circuits as ihc
import Qiskit_Circuits as qkc
import Q_Generator as QG
import Q_Loss_and_Gradients as Q_gradients
import Utils
import Discriminator_Utils as D_utils # A file that contains different discriminator architectures
import time

In [2]:
from quspin.operators import hamiltonian,exp_op # Hamiltonians and operators
from quspin.basis import spin_basis_1d # Hilbert space spin basis
from quspin.tools.misc import KL_div
from scipy import integrate,optimize,special,stats,interpolate
import numpy as np # generic math functions
from time import time # timing package
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Times New Roman"
matplotlib.rcParams.update({'font.size': 25})
#import tensorflow as tf
import cv2
from MMD_loss import RBFMMD2

ModuleNotFoundError: No module named 'MMD_loss'

In [ ]:
# IBMQ.enable_account("a66a83268fbf696a24b481c5d7568d50f14a1b2f28c6932d600a625b9cbffc6e28a6594bdd422676c9b2a214dcafa87d57012cc8678cbabaeecb319a0515f4c2")

In [ ]:
# provider = IBMQ.get_provider(hub='ibm-q')
# provider.backends()

In [ ]:
#backend = provider.get_backend('simulator_statevector')

In [ ]:
#======================================Start of Configurations=================================
n_qubits = 6 # We use 6 qubits, two hidden
n_h_qubits = 2
n_show_qubits = n_qubits-n_h_qubits
x_basis_m_n = torch.FloatTensor(Utils.binary_basis((n_show_qubits,))) 
sigma_list = [0.1,0.25,4,10]
K = Utils.mix_rbf_kernel(x_basis_m_n, x_basis_m_n, sigma_list)

geometry = (2,3) # The qubits are put on a 2 by 2 graph

#connections = q.get_nn_pairs((2,3)) # The qubits are CNOT-entangled by nearest neighbors on the 2 by 2 graph
connections = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5,0)]
print('connection scheme', connections)

simulator = qiskit.Aer.get_backend('statevector_simulator')

# t_m is the duration of each quench temperature, one delta_t time corresponds to one
# There is no quenche layers set up when you first call the circuit initialization:
circuit0 = qkc.MBL_Circuit(delta_t=0.1, t_m=1.0, Jxy=1.0, Jzz=1.0, n_qubits=n_qubits, n_h_qubits=n_h_qubits, 
                           connections=connections, 
                           backend=simulator, shots=100, if_measure=False)

circuit0._circuit.B_gate, circuit0._circuit.C_gate = circuit0.create_B_C()

connection scheme [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]


In [ ]:
# There is no quenche layers set up when you first call the circuit initialization.
# In this example, we start from the domain wall state |111000>
circuit0._circuit.draw()

In [ ]:
#======================================Create p_data==========================================
hdim = 2**n_show_qubits
p_data = torch.FloatTensor(Utils.gaussian_pdf((n_show_qubits,), hdim/2., hdim/4.))
plt.plot(p_data, '*')
plt.title('target distribution')


Fast: training loop with saved statevector at each quench

In [ ]:
M = 100 # Total number of quenches
n_branches = 500 # number of search branches at each quench
hd = 20 # Range of the randomly selected localization conef [-hd, +hd]
training_losses = [] # To record the best mmd out of all candidate points at each quench m

# Prepare the initial domain wall state
circuit0.init_wall_state()
circuit0.t_qc = transpile(circuit0._circuit, # Should transpile this whenever the circuit structure is altered
                         circuit0.backend)
qobj = qiskit.assemble(circuit0.t_qc)
job = circuit0.backend.run(qobj)
state = job.result().get_statevector() # This returns a list of complex numbers for the amps
print('wallstate', state)



for m in range(M):
    t0 = time.time()
    circuit0._circuit.data.clear() # clear the previous layers
    circuit0._circuit._parameter_table.clear() # clear parameter table
    #print('after clear', circuit0._circuit.data)
    # The init state will be the wall state, the intermediate state will the best quenced state at quench m
    circuit0._circuit.initialize(state, [i for i in range(n_qubits)])
    # This will automatically transpile:
    circuit0.append_layers_m(m=0, keep_previous=False) # since we will save the state vector after each quench. automatically transpile
    
    
    # First randomly select candidate configurations of hz for quench m:
    mmds = []
    theta_list_array = []
    for b in range(n_branches):
        # Only randomly earch prameterse for this specific quench
        theta_list_m = (torch.rand( circuit0.n_qubits )*2-1)*hd
        # use these parameters 
        px, _ = circuit0.pdf_actual(theta_list_m)
        mmd_distance = px@K@(p_data.T)
        mmds.append(mmd_distance)
        theta_list_array.append(theta_list_m) # keep a record of this for debugging
    
    #print('branch mmd', mmds)
    ind = np.argmin(mmds)
    print('best mmd', mmds[ind])
    training_losses.append(mmds[ind])
    # Recalculate the best evolved state for the next quench
    _, state = circuit0.pdf_actual(theta_list_array[ind])
    t1 = time.time()
    print('time', t1-t0)

In [ ]:
theta_list_m

In [ ]:
plt.plot(mmds)

In [ ]:
circuit0.thetas

In [ ]:
phi = qiskit.circuit.Parameter('phi')
phi2 = qiskit.circuit.Parameter('phi')
qc = qiskit.QuantumCircuit(1)
print('qc first praemeter', qc._parameter_table)
# parameterize the rotation
qc.rx(phi, 0)
print(qc.data)
qc.draw()

# bind the parameters after circuit to create a bound circuit
bc = qc.bind_parameters({phi: 3.14})
bc.measure_all()
bc.draw()
qc.rx(phi,0)
qc.rx(phi,0)



In [ ]:
A = []
theta = qiskit.circuit.Parameter('t')
A.append(theta)

In [ ]:
theta in A

In [ ]:
theta1 = qiskit.circuit.Parameter('t')
theta1 in A

In [ ]:
theta2 = copy.deepcopy(theta)
theta2 in A

In [ ]:
id(theta2)

In [ ]:
id(theta)

In [ ]:
id(theta1)

In [ ]:
theta2.name='new-t'

In [ ]:
qc._parameter_table

phi5 = copy.deepcopy(phi4)

In [ ]:
phi5 in qc._parameter_table

In [ ]:
phi3=phi# pass by reference
import copy
phi4=copy.deepcopy(phi)
qc.rx(phi4,0)# meaning that qiskist doesn't detect parameter duplicatres via address

In [ ]:
psi0=qiskit.circuit.Parameter('psi0')
qc._update_parameter_table(qc.data[0][0])

In [ ]:
qc.rx(phi2,0)

In [ ]:
qc.data

In [ ]:
phi4==phi

In [ ]:
phi.parameters

In [ ]:
qc.rx(phi2,0)

In [ ]:
qc._parameter_table.clear()

In [ ]:
qc.rx(phi2,0)

In [ ]:
circuit0._circuit.data

In [ ]:
circuit0._circuit.draw()

In [ ]:
torch.cat([torch.Tensor(), torch.zeros((3))])

In [ ]:
torch.ones(3)@torch.ones((3,3))@torch.ones(3)

In [ ]:
(torch.rand( (circuit0.current_m+1)*circuit0.n_qubits )*2-1)*1.8